# <center>Data Mining project: Discover and describe areas of interest<br> and events from geo-located parsed_data</center>

## #0: Import Dataset and Libraries

## Setting Environment

In [1]:
! python -m venv dataMiningEnv
#Activate windows
#! dataMiningEnv\Scripts\activate.bat

#Activate mac
! source dataMiningEnv/bin/activate


Actual environment location may have moved due to redirects, links or junctions.
  Requested location: "h:\IF4\fouilleD\Data-Lyon-Mining_4IF\dataMiningEnv\Scripts\python.exe"
  Actual location:    "\\ps-home.insa-lyon.fr\users\home\dlarrazmar\IF4\fouilleD\Data-Lyon-Mining_4IF\dataMiningEnv\Scripts\python.exe"
'source' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


## Importing Libraries

- ### Installs

In [2]:
# requires ipkernel
! pip install --upgrade pip'
# installation of required libraries and dependencies
# numeric calculations
! pip install numpy==1.26.0 
# data frames 
! pip install pandas==2.1.1 
# machine learning algorithms 
! pip install scikit-learn==1.5.1 
#! pip install scipy==1.12.0
# plotting 
#! pip install plotly==5.24.1 
! pip install matplotlib==3.8.0 
#! pip install seaborn==0.13.2 
#! pip install plotly-express==0.4.1 
#! pip install chart-studio==1.1.0 
# web app library 
#! pip install streamlit==1.37.1 
# association rules
#! pip install mlxtend==0.23.3
# Language processing
#! pip install nltk
#! python -m nltk.downloader popular # popular functions
# Folium
! pip install folium==0.12.1


Defaulting to user installation because normal site-packages is not writeable


ERROR: Invalid requirement: "pip'"
You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.26.0-cp39-cp39-win_amd64.whl (15.8 MB)


You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-2.1.1-cp39-cp39-win_amd64.whl (10.8 MB)
  Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)


...

Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_learn-1.5.1-cp39-cp39-win_amd64.whl (11.0 MB)
  Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
  Using cached scipy-1.13.1-cp39-cp39-win_amd64.whl (46.2 MB)


You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable

You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.



  Using cached matplotlib-3.8.0-cp39-cp39-win_amd64.whl (7.6 MB)
  Using cached kiwisolver-1.4.7-cp39-cp39-win_amd64.whl (55 kB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
  Using cached fonttools-4.55.3-cp39-cp39-win_amd64.whl (2.2 MB)
  Using cached pillow-11.1.0-cp39-cp39-win_amd64.whl (2.6 MB)
  Using cached pyparsing-3.2.1-py3-none-any.whl (107 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl (37 kB)
  Using cached contourpy-1.3.0-cp39-cp39-win_amd64.whl (211 kB)


### Imports

In [3]:
import pandas as pd
import re
from sklearn.preprocessing import StandardScaler
import sklearn.cluster as cl
import folium
from folium.plugins import MarkerCluster
#import nltk


## Importing Data

In [4]:
# Charger les données avec low_memory=False pour éviter les avertissements
data = pd.read_csv("data/flickr_data2.csv", sep=",", low_memory=False)

## #1: Discovering areas of interests using clustering

## Understanding the Data

--Write something to describe this part of the report--

## Data clearing and preparation

In [ ]:
# Print details about parsing step:
# -> Number of lines parsed - % of original data parsed
def parse_conclusion(parsed_data):
    l = len(parsed_data)
    print(f"<Lines parsed: {l} - {round(100*l/len(data),3)}% of original data>")

In [4]:
# Étape 1 : Nettoyage initial des colonnes
# Supprimer les espaces supplémentaires dans les noms des colonnes
parsed_data = data
parsed_data.columns = parsed_data.columns.str.strip()

# Convertir les colonnes temporelles en numériques
# Liste des colonnes temporelles
time_columns = [
    'date_taken_minute', 'date_taken_hour', 'date_taken_day',
    'date_taken_month', 'date_taken_year',
    'date_upload_minute', 'date_upload_hour', 'date_upload_day',
    'date_upload_month', 'date_upload_year'
]

# Convertir chaque colonne en int64, remplacer les erreurs par 0
for col in time_columns:
    parsed_data[col] = pd.to_numeric(data[col], errors='coerce').fillna(0).astype('int64')

In [ ]:
# Étape 2 : Suppression des doublons basés sur l'identifiant unique
# Sauvegarder les doublons pour audit futur
duplicate_data = parsed_data[parsed_data['id'].duplicated(keep='first')].sort_values("id")
duplicate_data.to_csv(
    "data/parsed_lines/duplicatedId.csv", index=False
)

# Supprimer les doublons
parsed_data = parsed_data[~parsed_data['id'].duplicated(keep='first')]

parse_conclusion(duplicate_data)

In [6]:
# Étape 3 : Gestion des colonnes inutilisées ou corrompues
# Identifier les colonnes inutiles
unused_columns = ["Unnamed: 16", "Unnamed: 17", "Unnamed: 18", 
                  "date_upload_minute", "date_upload_hour", "date_upload_day", 
                  "date_upload_month", "date_upload_year"]

# Sauvegarder les données corrompues
corrupted_data = parsed_data[parsed_data[unused_columns].notnull().any(axis=1)]
corrupted_data.to_csv(
    "data/parsed_lines/corrupted_data.csv"
, index=False)


# Supprimer les colonnes inutilisées et les lignes corrompues
parsed_data = parsed_data[~(parsed_data["Unnamed: 16"].notnull() | parsed_data["Unnamed: 17"].notnull() | parsed_data["Unnamed: 18"].notnull())]
parsed_data = parsed_data.drop(columns=unused_columns)

parse_conclusion(corrupted_data)

SyntaxError: invalid syntax (4018163999.py, line 13)

In [7]:
# Étape 4 : Nettoyage des coordonnées GPS
# Définir les limites géographiques de Lyon
lyon_lat_min, lyon_lat_max = 45.69, 45.85
lyon_lon_min, lyon_lon_max = 4.78, 4.92

# Sauvegarder les données not Lyonnaises ou non definies
out_lyon_data = parsed_data[
    ~((parsed_data['lat'] >= lyon_lat_min) & 
    (parsed_data['lat'] <= lyon_lat_max) &
    (parsed_data['long'] >= lyon_lon_min) &
    (parsed_data['long'] <= lyon_lon_max))
]
out_lyon_data.to_csv(
    "data/parsed_lines/out_lyon.csv"
, index=False)

# Filtrer les données pour garder uniquement les points dans Lyon
parsed_data = parsed_data[
    (parsed_data['lat'] >= lyon_lat_min) & 
    (parsed_data['lat'] <= lyon_lat_max) &
    (parsed_data['long'] >= lyon_lon_min) &
    (parsed_data['long'] <= lyon_lon_max)
]

parse_conclusion(out_lyon_data)

In [10]:
# Étape 5 : Gestion des valeurs manquantes
# Supprimer les lignes avec des valeurs manquantes
#parsed_data = parsed_data.dropna(subset=['id', 'user', 'lat', 'long', 'tags', 'title', 'date_taken_minute', 'date_taken_hour', 'date_taken_day', 'date_taken_month', 'date_taken_year'])

# Sauvegarder les lignes à données manquantes
val_manquante_data = parsed_data[parsed_data.drop(columns=['tags','title']).isna().any(axis = 1)]
val_manquante_data.to_csv(
    "data/parsed_lines/NaNs.csv"
, index=False)

# Remplir les valeurs manquantes pour les colonnes textuelles par des chaînes vides
parsed_data['tags'] = parsed_data['tags'].fillna('')
parsed_data['title'] = parsed_data['title'].fillna('')

parse_conclusion(val_manquante_data)

In [24]:
# Étape 6 : Filtrage des dates incohérentes

# Sauvegarder les données ayant des date non correcte
uncorrect_date_data = parsed_data[
    ~((parsed_data['date_taken_year'] >= 2009) & (parsed_data['date_taken_year'] <= 2025) &
    (parsed_data['date_taken_month'] >= 1) & (parsed_data['date_taken_month'] <= 12) &
    (parsed_data['date_taken_day'] >= 1) & (parsed_data['date_taken_day'] <= 31) &
    (parsed_data['date_taken_hour'] >= 0) & (parsed_data['date_taken_hour'] <= 23) &
    (parsed_data['date_taken_minute'] >= 0) & (parsed_data['date_taken_minute'] <= 59))
]
uncorrect_date_data.to_csv(
    "data/parsed_lines/uncorrect_date.csv"
, index=False)

# Garder uniquement les dates raisonnables (entre 2009 et 2025)
parsed_data = parsed_data[
    (parsed_data['date_taken_year'] >= 2009) & (parsed_data['date_taken_year'] <= 2025) &
    (parsed_data['date_taken_month'] >= 1) & (parsed_data['date_taken_month'] <= 12) &
    (parsed_data['date_taken_day'] >= 1) & (parsed_data['date_taken_day'] <= 31) &
    (parsed_data['date_taken_hour'] >= 0) & (parsed_data['date_taken_hour'] <= 23) &
    (parsed_data['date_taken_minute'] >= 0) & (parsed_data['date_taken_minute'] <= 59) 
]

parse_conclusion(uncorrect_date_data)

In [27]:
# Étape 7 : Nettoyage/Standarisation des colonnes textuelles
# Fonction pour nettoyer le texte
def clean_text(text):
    # Supprimer les caractères spéciaux et passer en minuscules
    return re.sub(r'[^a-zA-Z0-9, ]', '', text).lower()

# Appliquer le nettoyage sur les colonnes textuelles
parsed_data['tags'] = parsed_data['tags'].apply(clean_text)
parsed_data['title'] = parsed_data['title'].apply(clean_text)

In [ ]:
# Résumé final des données nettoyées
print("Données nettoyées :")
print(parsed_data.head(5))

# Sauvegarder les données nettoyées pour les prochaines étapes
parsed_data.to_csv("data/cleaned_flickr_data.csv", index=False)

Données nettoyées :
<class 'pandas.core.frame.DataFrame'>
Index: 388524 entries, 0 to 420239
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  388524 non-null  int64  
 1   user                388524 non-null  object 
 2   lat                 388524 non-null  float64
 3   long                388524 non-null  float64
 4   tags                388524 non-null  object 
 5   title               388524 non-null  object 
 6   date_taken_minute   388524 non-null  int64  
 7   date_taken_hour     388524 non-null  int64  
 8   date_taken_day      388524 non-null  int64  
 9   date_taken_month    388524 non-null  int64  
 10  date_taken_year     388524 non-null  int64  
 11  date_upload_minute  388524 non-null  int64  
 12  date_upload_hour    388524 non-null  int64  
 13  date_upload_day     388524 non-null  int64  
 14  date_upload_month   388524 non-null  int64  
 15  date_upload_year   

# Visualize geolocated points on a map

In [ ]:
# Define the center of Lyon
lyon_lat, lyon_lon = 45.75, 4.85  # Approximate coordinates for Lyon, France

# Create a Folium map
map_lyon = folium.Map(location=[lyon_lat, lyon_lon], zoom_start=12)

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(map_lyon)

# Add points from the dataset
for _, row in parsed_data.iterrows():
    folium.Marker(
        location=[row['lat'], row['long']],  # Replace with 'lat' and 'long' if necessary
        popup=f"Tags: {row['tags']}, Title: {row['title']}",  # Optional popup information
    ).add_to(marker_cluster)

# Display the map
map_lyon

# Optional: Save the map as an HTML file
map_lyon.save("lyon_map.html")